In [6]:
%load_ext autoreload
%autoreload 2

from deltas.pipeline import data, classifier, evaluation
from deltas.model import base, downsample
import deltas.plotting.plots as plots
from tqdm import tqdm

import numpy as np
import pandas as pd
# np.random.seed(10)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

N1 = 1000
N2 = 10
m = 1
v = 1
costs = (1, 1)  # change for (1, 10) to increase results
# Gaussian (not always seperable)
dfs = []
for i in tqdm(range(10)):
    data_clf = data.get_data(
        m1=[-m, -m],
        m2=[m, m],
        cov1=[[v, 0], [0, v]],
        cov2=[[v, 0], [0, v]],
        N1=N1,
        N2=N2,
        scale=False,
        test_nums=[100000, 1000],
    )

    model = 'SVM-linear'
    model = 'SVM-rbf'
    # model = 'Linear'
    model = 'MLP'

    classifiers_dict = classifier.get_classifier(
        data_clf=data_clf,
        model=model,
        _plot=False)
    data_clf['clf'] = classifiers_dict['Baseline']
    X = data_clf['data']['X']
    y = data_clf['data']['y']
    clf = data_clf['clf']
    # deltas_model = downsample.downsample_deltas(
    #     clf).fit(X, y, _print=True, _plot=True, max_trials=10000)
    # deltas_model = base.base_deltas(
    #     clf).fit(X, y, grid_search=True, _print=True, _plot=True)
    deltas_model = downsample.downsample_deltas(clf).fit(X, y,
                                                         alpha=10,
                                                         _print=False,
                                                         _plot=False,
                                                         method='supports-prop-update_mean',
                                                         max_trials=10000,
                                                         parallel=True)

    if deltas_model.is_fit == True:
        classifiers_dict['Our Method'] = deltas_model
        scores_df = evaluation.eval_test(classifiers_dict,
                                         data_clf['data_test'], _print=False, _plot=False)
        dfs.append(scores_df)
    else:
        print('not fit deltas')

 20%|██        | 2/10 [00:26<01:48, 13.59s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 30%|███       | 3/10 [00:39<01:30, 12.99s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 40%|████      | 4/10 [00:51<01:16, 12.73s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 60%|██████    | 6/10 [01:13<00:47, 11.76s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/s

not fit deltas


/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 80%|████████  | 8/10 [01:34<00:22, 11.21s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 10/10 [02:00<00:00, 12.07s/it]


In [8]:
print(len(dfs))

9


In [9]:
df = pd.concat(dfs, axis=0)
mean = {}
std = {}
index = df.index.unique().to_list()
cols = df.columns.to_list()
for method in index:
    mean[method] = df.loc[method].mean().to_list()
    std[method] = df.loc[method].std().to_list()

mean_df = pd.DataFrame.from_dict(mean, orient='index', columns=cols)
print(mean_df)
std_df = pd.DataFrame.from_dict(std, orient='index', columns=cols)

m = mean_df.to_dict('list')
s = std_df.to_dict('list')
metrics = mean_df.columns.to_list()
methods = mean_df.index.to_list()
sf = 5
for metric in metrics:
    means = m[metric]
    sts = s[metric]
    mx = np.argmax(means)
    for i in range(len(means)):
        m_str = str(means[i])[1:sf]
        if i == mx:
            m_str = f"\\textbf{{{m_str}}}"
        s_str = str(sts[i])[1:sf-1]
        m[metric][i] = f'${m_str} \\pm {s_str}$'

method_map = {
    'Baseline': 'Baseline',
    'SMOTE': "SMOTE \cite{Chawla_2002_JAIR}",
    'Balanced Weights': 'BW',
    'BMR': 'BMR \cite{Bahnsen_2014_SIAM}',
    'Threshold': 'Thresh \cite{Sheng_2006_AAAI}',
    'Our Method': 'Our Method',
}
meths_new = []
for me in methods:
    meths_new.append(method_map[me])
m['Methods'] = meths_new
df = pd.DataFrame(m)  # .set_index('Methods')
meths = df.pop('Methods')
df.insert(0, 'Methods', meths)
latex_str = df.to_latex(index=False)

print('\\begin{tabular}{@{}lrrrr@{}}'+latex_str[22:])

                  Accuracy    G-Mean   ROC-AUC        F1
Baseline          0.991272  0.324197  0.968552  0.217000
SMOTE             0.956963  0.850874  0.954326  0.287302
Balanced Weights  0.973527  0.805125  0.955167  0.342804
BMR               0.954158  0.870383  0.968552  0.318332
Threshold         0.954158  0.870383  0.968552  0.318332
Our Method        0.939452  0.888485  0.968552  0.262270
\begin{tabular}{@{}lrrrr@{}}
\toprule
Methods & Accuracy & G-Mean & ROC-AUC & F1 \\
\midrule
Baseline & $\textbf{.991} \pm .00$ & $.324 \pm .17$ & $\textbf{.968} \pm .00$ & $.216 \pm .15$ \\
SMOTE \cite{Chawla_2002_JAIR} & $.956 \pm .01$ & $.850 \pm .05$ & $.954 \pm .03$ & $.287 \pm .10$ \\
BW & $.973 \pm .00$ & $.805 \pm .09$ & $.955 \pm .03$ & $\textbf{.342} \pm .07$ \\
BMR \cite{Bahnsen_2014_SIAM} & $.954 \pm .02$ & $.870 \pm .03$ & $.968 \pm .00$ & $.318 \pm .14$ \\
Thresh \cite{Sheng_2006_AAAI} & $.954 \pm .02$ & $.870 \pm .03$ & $.968 \pm .00$ & $.318 \pm .14$ \\
Our Method & $.939 \pm .0

In [10]:
df.loc['BMR'].mean().to_list()
df.columns.to_list()

KeyError: 'BMR'